In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/smollm2

Mounted at /content/drive
/content/drive/MyDrive/smollm2


In [ ]:
#!pip install -q -U torch=='2.0.0'
!pip install "torch==2.5.0"

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install  --upgrade  "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.23.2" "trl==0.8.6" "peft==0.10.0"


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4qj5lnxe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-4qj5lnxe
  Resolved https://github.com/huggingface/transformers to commit 3ea3ab62d80d91f9bdd16bd3cacd8133fb0d4566
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10053105 sha256=0464ecbd67adcefd978e6448f2f321db0e63d8c83d68d55eb9274010ca5d80fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-z__4mj4x/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully 

In [ ]:
import pandas as pd


data_type='Original'
#data_type='Synthetic'

# Load Data
df = pd.read_excel('LR_Dataset_Original_Sythetic_Final.xlsx')
#df = pd.read_excel('LR_Dataset_Original_Sythetic_Experiment_70.xlsx') # 20% of the training and validation sets


if data_type=='Original':
 df = df[df['Source'] == 'Original'] # Only original

df= df[['Sentence','Category' ,'Classification']]

df['Sentence'] = df['Sentence'].str.capitalize()
df=df.sample(frac=1).reset_index(drop=True)
df['Sentence'] = (df['Sentence']
                  .str.strip()
                  .str.replace(r'\n|\r', ' ', regex=True)
                  .str.replace(r'\s{2,}', ' ', regex=True))
df

,Sentence,Category,Classification
0,Search strategies include reinforcement learni...,OVERALL,TEST
1,Two meta-analyses have identified significant ...,RESULT,TRAINING
2,"However, this previous study was conducted wit...",LIMITATION,TRAINING
3,Our approach is inspired by richer type level ...,EXTENSION,TEST
4,Some researchers using cfa have found that the...,RESULT,TRAINING
...,...,...,...
695,Music recommendation is a popular research top...,OVERALL,VALIDATION
696,"One threat, also discussed by hellendoorn et a...",LIMITATION,TRAINING
697,"As far as we know, only li et al. [21] propose...",DESCRIPTION,TRAINING
698,The study’s results confirmed that this combin...,RESULT,TRAINING


In [ ]:
def create_instruction(row):
    sentence = row['Sentence']
    instruction = (
                "You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\n"
                "Categories:\n "
                "1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n "
                "2. RESEARCH GAP: Highlights the need for further research within the topic.\n"
                "3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n"
                "4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “found”, “demonstrated”, and “observed” or phrases like “the findings indicate”.\n"
                "5. LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study that hinders generalizability or reliability in a previous study.\n"
                "6. EXTENSION: Describes how the current study addresses or extends previous studies by stating the overall idea, contrasting ideas or elaborating further ideas. It usually uses the words “we” or “our”.\n"
                "7. OTHER: Any text that does not fit the above categories.\n"
                "Procedure:\n"
                "1. Determine whether the subject of the sentence is a topic or a study.\n"
                "2. Identify the most suitable category based on the content. Do not create new categories. Use the categories given above.\n"
                "3. Provide the category number that best fits the sentence. Just provide the category number without any explanation.\n"

                f"Sentence: {sentence}.\n"
            )

    return instruction

In [ ]:
df['instruction'] = df.apply(create_instruction, axis=1)

df = df.rename(columns={'Category': 'response'})

df

,Sentence,response,Classification,instruction
0,Search strategies include reinforcement learni...,OVERALL,TEST,You are a researcher that should assign a clas...
1,Two meta-analyses have identified significant ...,RESULT,TRAINING,You are a researcher that should assign a clas...
2,"However, this previous study was conducted wit...",LIMITATION,TRAINING,You are a researcher that should assign a clas...
3,Our approach is inspired by richer type level ...,EXTENSION,TEST,You are a researcher that should assign a clas...
4,Some researchers using cfa have found that the...,RESULT,TRAINING,You are a researcher that should assign a clas...
...,...,...,...,...
695,Music recommendation is a popular research top...,OVERALL,VALIDATION,You are a researcher that should assign a clas...
696,"One threat, also discussed by hellendoorn et a...",LIMITATION,TRAINING,You are a researcher that should assign a clas...
697,"As far as we know, only li et al. [21] propose...",DESCRIPTION,TRAINING,You are a researcher that should assign a clas...
698,The study’s results confirmed that this combin...,RESULT,TRAINING,You are a researcher that should assign a clas...


In [ ]:
test_dataset =df[df['Classification'] == 'TEST']
test_dataset

,Sentence,response,Classification,instruction
0,Search strategies include reinforcement learni...,OVERALL,TEST,You are a researcher that should assign a clas...
3,Our approach is inspired by richer type level ...,EXTENSION,TEST,You are a researcher that should assign a clas...
14,Mcauley et al. [21] mine key attributes from t...,DESCRIPTION,TEST,You are a researcher that should assign a clas...
26,To improve the ability of preference-extractor...,EXTENSION,TEST,You are a researcher that should assign a clas...
28,In order to reflect the users complete spectru...,DESCRIPTION,TEST,You are a researcher that should assign a clas...
...,...,...,...,...
674,Leveraging whole expressions instead of only w...,DESCRIPTION,TEST,You are a researcher that should assign a clas...
675,"However, they need to manually extract the 2d ...",LIMITATION,TEST,You are a researcher that should assign a clas...
687,"This is a concerning gap in the literature, gi...",RESEARCH GAP,TEST,You are a researcher that should assign a clas...
691,Our study complements the second mapping study...,EXTENSION,TEST,You are a researcher that should assign a clas...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline

# Model Setup (Quantization and Tokenizer)

model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, # This new
    device_map="auto",
    trust_remote_code=True,
)

# Tokenizer Initialization
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_bos_token=True,
    trust_remote_code=True,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "smollm2_7B_Instruct-lro-finetune")

In [ ]:
def generate_test_prompt(example):
    """Format prompt for training."""
    text = f"<|im_start|>user\n{example['instruction']}<|im_end|>"
    return text

In [ ]:
test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


<ipython-input-11-ca85691956fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


In [ ]:
test_dataset

,Sentence,response,Classification,instruction,text
0,Search strategies include reinforcement learni...,OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
3,Our approach is inspired by richer type level ...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
14,Mcauley et al. [21] mine key attributes from t...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
26,To improve the ability of preference-extractor...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
28,In order to reflect the users complete spectru...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
...,...,...,...,...,...
674,Leveraging whole expressions instead of only w...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
675,"However, they need to manually extract the 2d ...",LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
687,"This is a concerning gap in the literature, gi...",RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
691,Our study complements the second mapping study...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...


In [ ]:
def extract_text(text):
    # Define both markers to search for
    markers = ["<|im_end|>assistant", "assistant\n"]

    # Loop through markers and check if each is in the text
    for marker in markers:
        marker_position = text.find(marker)

        # If the marker is found, extract text after it
        if marker_position != -1:
            return text[marker_position + len(marker):].strip()  # Remove any leading/trailing whitespace

    # Return None if neither marker is found
    return text

In [ ]:
def clean_text(text):

    # Check for keywords and replace text accordingly
    if "OVERALL" in text:
        return "OVERALL"
    if "RESEARCH GAP" in text:
        return "RESEARCH GAP"
    if "DESCRIPTION" in text:
        return "DESCRIPTION"
    if "RESULT" in text:
        return "RESULT"
    if "LIMITATION" in text:
        return "LIMITATION"
    if "EXTENSION" in text:
        return "EXTENSION"
    elif "OTHER" in text:
        return "OTHER"

    # If neither keyword is found, return the original text
    return text

In [ ]:
prompt=test_dataset.iloc[0]['text']
prompt


'<|im_start|>user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “sh

In [ ]:
inputs = tokenizer(prompt,
                return_tensors="pt"
            ).to("cuda")

outputs = ft_model.generate(**inputs, max_new_tokens=15, use_cache=True)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

answer

'user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “foun

In [ ]:
clean_text(extract_text(answer))

'OVERALL'

In [ ]:
#pipe = pipeline(task="text-generation",
#                        model=ft_model,
#                        tokenizer=tokenizer,
#                        max_new_tokens = 100,
#                        temperature = 0.0,
#                       )
#result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
#answer = result[0]['generated_text']

In [ ]:
def get_classification_finetuning(data_point,model,tokenizer):
    """
    Gets the classification for a data point using the fine-tuned model.
    """
    inputs = tokenizer(data_point['text'],return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=15, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(clean_text(extract_text(answer)))
    data_point['Prediction_Finetune']=answer  # Assign the result to the data point
    data_point['Prediction_Finetune_Cleaned']=clean_text(extract_text(answer))

    return data_point
# Apply the get_classification function to the dataset using map
test_dataset = test_dataset.apply(lambda row: get_classification_finetuning(row, ft_model, tokenizer), axis=1)

OVERALL
EXTENSION
DESCRIPTION
EXTENSION
DESCRIPTION
OVERALL
RESULT
RESULT
LIMITATION
RESEARCH GAP
LIMITATION
LIMITATION
RESULT
OTHER
OTHER
LIMITATION
RESULT
EXTENSION
EXTENSION
OVERALL
RESULT
DESCRIPTION
EXTENSION
OVERALL
RESEARCH GAP
RESULT
RESEARCH GAP
DESCRIPTION
RESEARCH GAP
RESULT
OVERALL
OVERALL
RESEARCH GAP
OVERALL
RESULT
OVERALL
EXTENSION
OVERALL
LIMITATION
EXTENSION
EXTENSION
RESULT
RESEARCH GAP
RESEARCH GAP
OTHER
DESCRIPTION
RESEARCH GAP
OTHER
EXTENSION
EXTENSION
DESCRIPTION
OTHER
OVERALL
EXTENSION
LIMITATION
OTHER
RESULT
RESULT
OTHER
EXTENSION
RESEARCH GAP
EXTENSION
OTHER
RESULT
OTHER
DESCRIPTION
OVERALL
DESCRIPTION
OVERALL
RESEARCH GAP
LIMITATION
OVERALL
LIMITATION
DESCRIPTION
RESEARCH GAP
EXTENSION
OVERALL
RESEARCH GAP
OVERALL
OTHER
DESCRIPTION
RESULT
DESCRIPTION
RESEARCH GAP
RESEARCH GAP
OTHER
DESCRIPTION
OTHER
OTHER
RESULT
LIMITATION
OTHER
OVERALL
RESULT
EXTENSION
RESEARCH GAP
RESULT
LIMITATION
DESCRIPTION
OTHER
EXTENSION
DESCRIPTION
DESCRIPTION
OVERALL
LIMITATION
OTHER


In [ ]:
test_dataset


,Sentence,response,Classification,instruction,text,Prediction_Finetune,Prediction_Finetune_Cleaned
0,Search strategies include reinforcement learni...,OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,OVERALL
3,Our approach is inspired by richer type level ...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,EXTENSION
14,Mcauley et al. [21] mine key attributes from t...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,DESCRIPTION
26,To improve the ability of preference-extractor...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,EXTENSION
28,In order to reflect the users complete spectru...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,DESCRIPTION
...,...,...,...,...,...,...,...
674,Leveraging whole expressions instead of only w...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,DESCRIPTION
675,"However, they need to manually extract the 2d ...",LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,LIMITATION
687,"This is a concerning gap in the literature, gi...",RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,RESEARCH GAP
691,Our study complements the second mapping study...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,EXTENSION


In [ ]:
#test_dataset.to_csv('SmolLM2_FT_Test_LoRA_Augmented_Final.csv', index=False)
test_dataset.to_csv('SmolLM2_FT_Test_NEFT_Augmented3.csv', index=False)
